# Homework 2


## Mc-Call Job Search Model with Separation

We consider here a very simple job-search model, with separation. Our goal consists in solving this model using a policy iteration algorithm. Some background on the McCall and numerical ideas are discussed on the [QuantEcon](`https://julia.quantecon.org/mccall_model.html`) website.

There is a single worker who can be either employed ("e") or unemployed ("u") in any period.

When unemployed, the jobless worker receives unemployment benefits $c_t=\alpha>0$ in every period as long as he stays unemployed. He also receives a salary offer $w_t$ which is drawn from a discrete i.i.d. distribution and takes values $w_1, ..., w_K$ with probabilities $p_1, ... p_K$ respectively.

When an unemployed worker accepts an offer in period $t$, he gets the salary $w_t$ and becomes employed. He then keeps his salary $w_t$ as long as he stays employed (for $s\geq t$, $c_s=w_t$ if $t$ is the date at which worker got the current job); in each period he has a probability $\lambda$ of becoming unemployed in the next period and remains employed otherwise.

When a worker receives a given amount $x$ his perceived utility is $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ with $\gamma>1.0$. A worker discounts the future at a rate $\beta \in [0,1[$. As a result, in any period $t_0$ workers seek to maximize $\sum_{t\geq t_0}^{\infty} U(c_t)$.


__Define a parameter type `Parameter`, with fields $\alpha$, $\beta$, $\gamma$, $K$, $\sigma$, $\lambda$. Create a parameter variable $\omega$ with $\alpha=0.5$, $\beta=0.96$, $\gamma=4$, $K=10$, $\sigma=0.6$, $\lambda=0.015$__

In [19]:
struct Parameter{α,β,γ,K,σ,λ}
    α::α
    β::β
    γ::γ
    K::K
    σ::σ
    λ::λ
end

In [20]:
ω = Parameter(0.5,0.96,4,10,0.6,0.015)

Parameter{Float64,Float64,Int64,Int64,Float64,Float64}(0.5, 0.96, 4, 10, 0.6, 0.015)

__What is the uniform distribution, whose mean is 1 and standard deviation is $\sigma$? Write a function `discrete_uniform(σ::Float64, K::Int64)::Tuple{Vector{Float64} Vector{Float64}}` to discretize it, using $K$ points. The function should return two vectors `w` and `p` of floats of the same size `K`. Check the results satisfy the right conditions (uniformity, standard deviation).__

We are looking for a uniform distribution $\mathcal{U}(a,b)$ such that the $mean$ is $1$ and the $std = \sigma$

- mean of uniform $\mathcal{U}(a,b) = \frac{a+b}{2} = 1$

- variance of uniform $\mathcal{U}(a,b) = \frac{(b-a)^{2}}{12} = \sigma^{2}$

By solving the system, we get two solutions: 

- (1) $(a_1,b_1) = (1+\sqrt{3}\sigma,1-\sqrt{3}\sigma)$

- (2) $(a_2,b_2) = (1-\sqrt{3}\sigma,1+\sqrt{3}\sigma)$

Since the parameters of the Uniform distribution are such that $a<b$, the uniform distribution $\mathcal{U}(a,b)$ such that the $mean$ is $1$ and the $std = \sigma$ is :
$\mathcal{U}(1-\sqrt{3}\sigma,1+\sqrt{3}\sigma)$

In [21]:
function discrete_uniform(σ::Float64,K::Int64)::Tuple{Vector{Float64},Vector{Float64}}
    w = [1-sqrt(3)*σ]
    s = 2*sqrt(3)*σ/K
    for i in 1:K
        append!(w,i*s+w[1])
    end
    p = [1/K for i in 1:K]
    return w,p
end

discrete_uniform (generic function with 1 method)

In [22]:
w,p = discrete_uniform(ω.σ,ω.K)

([-0.03923048454132627, 0.16861561236693898, 0.3764617092752042, 0.5843078061834694, 0.7921539030917347, 1.0, 1.2078460969082652, 1.4156921938165306, 1.6235382907247957, 1.8313843876330609, 2.039230484541326], [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])


The optimal decision of a worker is characterized by two value functions: $V^E(w)$ is the value of being employed at wage $w$ and $V^U(w)$ the value of being unemployed, while receiving job offer $w_t$. In Julia, both $V^U$ and $V^E$ will be represented by arrays `V_U` and `V_E` of size `K`.

A policy `g(w)` is a binary choice in the unemployed state: accept or reject an offer $w_t$. It will then naturally be represented by a boolean array (type `zeros(Bool, K)` to initialize one).

In [23]:
g(w)= zeros(Bool,ω.K)

g (generic function with 1 method)

__Given a policy $g$, write down the recursive equations which defines the corresponding value functions $V^{U,g}(w)$ and $V^{E,g}(w)$.__

$V^{E,g}(\omega_t) = u(\omega_{t}) + \beta[(1-\lambda)V^{E,g}(\omega_{t})+ \lambda \sum_{t+1}^{\infty} V^{U,g}(w_{t+1})\frac{1}{K}] $ 

$V^{U,g}(\omega_t) = \max{[V^{E,g}(\omega_{t}),u(\alpha) + \beta\sum_{t+1}^{\infty} V^{U,g}(w_{t+1})\frac{1}{K}]} $


In [24]:
V_U = rand(Float64,ω.K)
V_E = rand(Float64,ω.K)

10-element Array{Float64,1}:
 0.9669101435509513
 0.9525967924332666
 0.9437388558713744
 0.4794553512443507
 0.4054424366365992
 0.7955939033326183
 0.10857419656377654
 0.3381970938004992
 0.5993369400718971
 0.13240381891469144

__Write a function which takes a guess $V^{U,g}(w)$ and $V^{E,g}(w)$ and a policy function $g$ as arguments (and other model parameters) and updates the values, according to the updating equations. This function could have signature `value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})::Vector{Float64}` where the returned vector has the same size as the supplied ones.__

In [25]:
function uti(ω,x)
    return (x^(1-ω.γ))/(1-ω.γ)
end

uti (generic function with 1 method)

In [26]:
using Pkg; Pkg.add("LinearAlgebra")

   Updating registry at `~/.juliapro/JuliaPro_v1.4.0-1/registries/JuliaPro`
  Resolving package versions...
   Updating `~/.juliapro/JuliaPro_v1.4.0-1/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.juliapro/JuliaPro_v1.4.0-1/environments/v1.4/Manifest.toml`
 [no changes]


In [27]:
using LinearAlgebra

In [28]:
function value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})
    e = 0.001
    max_iter = 1000
    dist = 1
    k = 0
    V_unext = zeros(Float64,ω.K)
    V_enext = zeros(Float64,ω.K)
    while k < max_iter && dist > e
        for i in 1:ω.K
            V_enext[i] = uti(ω,w[i]) + ω.β*((1-ω.λ)*V_E[i]+ω.λ*V_U[i]*p[i])
            if V_enext[i] > uti(ω,ω.α)+ω.β*sum([V_U[i]*p[i] for i in 2:ω.K])
                V_unext[i] = V_enext[i]
            else
                V_unext[i] = uti(ω,ω.α)+ω.β*sum([V_U[i]*p[i] for i in 2:ω.K])
            end
        end
        dist = norm(V_enext-V_E)
        V_E = V_enext
        V_U = V_unext
        k = k+1
    end
    println("dist =",dist)
    println("iterations =",k)
    return V_E,V_U
end

value_update (generic function with 1 method)

In [ ]:
value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})


__Write a function `eval_policy(g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Tuple{Vector{Float64}, Vector{Float64}}` which iterates on `value_update` find the values that satisfy the evaluation equations for policy `g`.__


In [30]:
function eval_policy(V_U::Vector{Float64}, V_E::Vector{Float64},g::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)
    V_E,V_U = value_update(V_U::Vector{Float64}, V_E::Vector{Float64}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64})
    for i in 1:ω.K
        if V_E[i] > V_U[i]
            g[i] = 1
        else
            g[i] = 0
        end
    end
    return g
end

eval_policy (generic function with 1 method)


__Write a function `improve_policy(g::Vector{Bool}, V_U::Vector{Bool}, V_E::Vector{Bool}, ω::Parameter, w::Vector{Float64}, p::Vector{Float64}, η::Float64)::Vector{Float64}` which returns the improved policy given guesses for the value function(s) at for $t+1$.__

__Implement the policy function algorithm. Print the successive approximation errors and comment on the convergence speed.__


## Neoclassical growth model (2)

We consider here, another deterministic version of the neoclassical growth model, but propose a slightly different solution method.

A representative agent uses capital $k_t$ to produce $y_t$ using the following production function:

$$y_t = k_t^{\alpha}$$

He chooses to consume an amount $c_t \in ]0, y_t]$ and invests what remains:

$$i_t = y_t - c_t$$.

He accumulates capital $k_t$ according to:

$$k_{t+1} = \left( 1-\delta \right) k_{t} + i_{t}$$

where $\delta$ is the depreciation rate and $i_t$ is the amount invested.

The goal of the representative agent is to maximize:

$$\sum_{t\geq 0} \beta^t U(c_t)$$

where $U(x)=\frac{x^{1-\gamma}}{1-\gamma}$ and $\beta<1$ is the discount factor.

Since the problem is time homogenous, the value function depends on available capital only and satisfies the following Bellman equation:

$$V\left(\underbrace{k}_{k_t}\right) = \max_{c\in[0,1[} U(c) + \beta V\left(\underbrace{(1-\delta)k + \underbrace{(k^{\alpha}-c)}_{y_{t+1}}}_{k_{t+1}}\right)$$

Our goal is to obtain a smooth approximation of $k$ and $V$ by using interpolations techniques.

For this model, using the dynamic first-order conditions, one can show the deterministic steady-state of the model satisfies $1=\beta \left( (1-\delta) + \alpha k^{\alpha -1} \right)$. 

__Create a suitable Parameter type to hold the parameters. Write a function `steady_state(p::Parameter)` to compute the steady-state capital `kbar` and the corresponding steady-state consumption `cbar`__ 


In [1]:
struct Growth{α,β,γ,δ}
    alph::α
    bet::β
    gam::γ
    delt::δ
    end

In [2]:
p = Growth(0.5,0.3,0.2,0.03)

Growth{Float64,Float64,Float64,Float64}(0.5, 0.3, 0.2, 0.03)

In [3]:
function steady_state(p::Growth)
    return ((1-p.bet*(1-p.delt))/(bet*alph))^(1/(1-p.alph))
end

steady_state (generic function with 1 method)

__Set $N=10$ and define a reasonable grid `kgrid=range(kmin, kmax; length=N)` to approximate capital $k$.__


In [4]:
N = 10
kgrid = range(0,10;length=N)

0.0:1.1111111111111112:10.0

__The unknown value function is represented as a `N` elements arrays. Define `Vi(k,p)=U(δ * k^α)/(1-𝛽)` and compute the initial guess `V0 = [Vi(k,p) for k in kgrid]`. Define a finer grid `ktest=range(kmin, kmax;length=1000)` and find the values of `Vi` on it by  using `Interpolations.jl` library to interpolate `V0` between the points of `kgrid`.__


In [5]:
function util(p,x)
    return (x^(1-p.gam))/(1-p.gam)
end

util (generic function with 1 method)

In [6]:
Vi(k,p) = util(p,p.delt*k^p.alph)*(1/(1-p.bet))
V0 = [Vi(k,p) for k in kgrid]

10-element Array{Float64,1}:
 0.0
 0.11267095437146306
 0.1486702156074773
 0.17484792185017442
 0.19617152559037812
 0.2144865060681127
 0.23071321606350592
 0.24538679537821556
 0.2588498798848912
 0.2713369736314919

In [ ]:
using Pkg; Pkg.add("Interpolations")

In [11]:
using Interpolations

In [15]:
ktest = range(0,10,length=1000)

0.0:0.01001001001001001:10.0

In [17]:
itp = interpolate(V0,BSpline(Cubic(Line(OnGrid()))))
sitp = scale(itp,kgrid)

10-element scale(interpolate(OffsetArray(::Array{Float64,1}, 0:11), BSpline(Cubic(Line(OnGrid())))), (0.0:1.1111111111111112:10.0,)) with element type Float64:
 -3.469446951953614e-18
  0.11267095437146306
  0.14867021560747726
  0.1748479218501744
  0.19617152559037815
  0.21448650606811268
  0.23071321606350592
  0.2453867953782156
  0.2588498798848912
  0.2713369736314919


__Compute a Bellman improvement function `bellman(V0::Vector{Float64}, p::Parameter, kgrid)::Tuple{Vector{Float64}, Vector{Float64}}` which does the following steps:__

- take an initial guess `V0` for the value function

- at each grid point from kvec, optimize nonlinearly, the function $c \rightarrow U(c) + \beta V\left((1-\delta)k + (k^{\alpha}-c)\right)$ for each capital level in the grid `kvec`. In this expression the function `V()` interpolates `V0` defined on `kvec` on any point `k` so that the resulting function is continuous. 

- return the updated value and investment rules


__Write a value interation function `vfi(N, p)` which solves the model defined by parameter `p` using the value function algorithm. The function should return the value function and the policy rule.__


__Plot the solution. Comment.__

__Bonus 1: plot a graph showing the convergence back to the steady-state__


__Bonus 2: implement the policy iteration algorithm by adding an evaluation step in the `vfi` function.__
